# Preparazione dati sia storici che non

In [ ]:
import pandas as pd
import numpy as np
import pickle


In [ ]:
delivery_data = pd.read_csv('delivery_history.csv').copy()

Veloce overview del contenuto del csv

In [ ]:
delivery_data.info()
delivery_data.describe()
delivery_data.head()

## Conversione dei dati nei relativi tipi di dati specifici e controllare se tutti i valori sono stati convertiti correttamente

In [ ]:
# ✅ Conversione sicura di 'delivery_date'
delivery_date_parsed = pd.to_datetime(delivery_data['delivery_date'], errors='coerce').dt.date

# Trova valori non convertibili
invalid_delivery_dates = delivery_data.loc[delivery_date_parsed.isna(), 'delivery_date'].unique()
print("❌ Valori non convertiti in 'delivery_date':")
print(invalid_delivery_dates)

# Applica conversione solo se valida
delivery_data['delivery_date'] = delivery_date_parsed


In [ ]:
# ✅ Conversione sicura di 'is_event' a booleano
# Prima controlliamo che tutti i valori siano 0 o 1 (o equivalenti)
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])

# Stampa eventuali valori invalidi
invalid_is_event = delivery_data.loc[~valid_bool_mask, 'is_event'].unique()
print("\n❌ Valori non convertibili in booleani in 'is_event':")
print(invalid_is_event)

# Applica conversione solo ai validi (opzionale: puoi forzare tutto a bool, ma questo è più sicuro)
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)

# Filtra i dati per mantenere solo le righe con 'is_event' = True
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()


In [ ]:
# Step 1: tenta la conversione usando pd.to_datetime in modo vettorizzato
start_times = pd.to_datetime(delivery_data['window_start_0'], format='%H:%M:%S.%f', errors='coerce')
end_times = pd.to_datetime(delivery_data['window_end_0'], format='%H:%M:%S.%f', errors='coerce')

# Step 2: crea solo HH:MM (datetime.time con secondi e microsecondi azzerati)
delivery_data['window_start_0'] = start_times.dt.time.apply(lambda t: t.replace(second=0, microsecond=0) if pd.notnull(t) else None)
delivery_data['window_end_0'] = end_times.dt.time.apply(lambda t: t.replace(second=0, microsecond=0) if pd.notnull(t) else None)

# Step 3: estrai i valori che non sono stati convertiti (NaT → errore)
invalid_start = delivery_data.loc[start_times.isna(), 'window_start_0'].unique()
invalid_end = delivery_data.loc[end_times.isna(), 'window_end_0'].unique()

# Step 4: stampa risultati
print("❌ Valori non convertiti in window_start_0:")
print(invalid_start)

print("\n❌ Valori non convertiti in window_end_0:")
print(invalid_end)


Eliminazione dei sabati dai delivery data

In [ ]:
# Escludi sabati e domeniche in modo vettorizzato e compatto
delivery_data = delivery_data[delivery_data['delivery_date'].map(lambda d: d.weekday() < 5)].copy()

# Convertiamo la colonna data in datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# Drop righe con date non valide
delivery_data = delivery_data.dropna(subset=['delivery_date'])


In [ ]:
delivery_data.info()
delivery_data.describe()
delivery_data.head()

## Preparazione di altre variabili rappresentanti i vincoli del problema

In [ ]:
capacity = 2100  # Capacità massima del veicolo
depot_location = (45.399332372533415, 9.28379118387267)  # Coordinate del magazzino (latitudine, longitudine)
shift_duration = 8 * 60  # Durata massima del percorso in minuti (8 ore)

## Generare la lista di tutti i possibili punti di consegna

Generazione base di dati per il confronto agosto-settembre (AS) vs ottobre-novembre (ON)

In [ ]:
# Filtra solo i mesi agosto-settembre e ottobre-novembre

delivery_data_AS = delivery_data[delivery_data['delivery_date'].dt.month.isin([8, 9])]
delivery_data_ON = delivery_data[delivery_data['delivery_date'].dt.month.isin([10, 11])]
delivery_data_OND = delivery_data[delivery_data['delivery_date'].dt.month.isin([10, 11, 12])]

delivery_points_AS = delivery_data_AS[['location_id', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)
delivery_points_ON = delivery_data_ON[['location_id', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)
delivery_points_OND = delivery_data_OND[['location_id', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)

# eliminazione dei punti troppo lontani
delivery_points_AS = delivery_points_AS[(delivery_points_AS['location_id'] != 14930) & (delivery_points_AS['location_id'] != 15133)]
delivery_points_ON = delivery_points_ON[(delivery_points_ON['location_id'] != 14930) & (delivery_points_ON['location_id'] != 15133)]
delivery_points_OND = delivery_points_OND[(delivery_points_OND['location_id'] != 14930) & (delivery_points_OND['location_id'] != 15133)]


In [ ]:
# tutti i delivery point unici
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)


print("Punti di consegna unici trovati:", len(delivery_points))

# -----------------------------
# 2. Filtra i dati (elimina quelli troppo lontani)
# -----------------------------
delivery_points = delivery_points[(delivery_points['location_id'] != 14930) & (delivery_points['location_id'] != 15133)]  # rimuovi record troppo lontani


print("Dopo l'eliminazione di punti troppo distanti, sono rimasti", len(delivery_points), "punti di consegna")


In [ ]:
print(f"delivery points esempio:\n{delivery_points.head()}")

## Creazione della matrice delle distanze da utilizzare nell'OR-tool

Prende in input un singolo cluster e ne restituisce la matrice delle distanze (espresse in minuti di percorrenza) e due variabili utili per il mapping tra index e location_id

In [ ]:
def distance_matrix_creation(period, delivery_points_custom=None):

	# period può essere = 'AS' o 'ON' o 'OND' o qualsiasi altro valore (tutti i punti)
	
	# 1) Costruisco la lista "depot + punti"
	
	points = pd.DataFrame([{"location_id": 0, "lat": depot_location[0], "lon": depot_location[1]}])

	if delivery_points_custom is not None:
		points = pd.concat([points, delivery_points_custom[['location_id','lat','lon']]], ignore_index=True)
	elif period == 'AS':
		points = pd.concat([points, delivery_points_AS[['location_id','lat','lon']]], ignore_index=True)
	elif period == 'ON':
		points = pd.concat([points, delivery_points_ON[['location_id','lat','lon']]], ignore_index=True)
	elif period == 'OND':
		points = pd.concat([points, delivery_points_OND[['location_id','lat','lon']]], ignore_index=True)
	else:
		points = pd.concat([points, delivery_points[['location_id','lat','lon']]], ignore_index=True)
	

	# 2) Haversine vectorized -> matrice NxN in metri (int)
	R = 6371000.0  # raggio medio terrestre in metri

	lat = np.radians(points['lat'].values)   # shape (N,)
	lon = np.radians(points['lon'].values)   # shape (N,)

	# broadcasting: differenze tutte-vs-tutte
	dlat = lat[:, None] - lat[None, :]
	dlon = lon[:, None] - lon[None, :]

	a = np.sin(dlat/2.0)**2 + np.cos(lat)[:, None] * np.cos(lat)[None, :] * np.sin(dlon/2.0)**2
	c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))
	distance_matrix_metri = (R * c).astype(int)  # metri

	# trasformo le distanze in tempi di percorrenza in minuti

	# 1.1) metri -> km
	dist_km = distance_matrix_metri.astype(float) / 1000.0

	# 2.1) modello di velocità (km/h) per fasce di distanza
	#    [0–5] km -> 30 km/h  |  (5–20] km -> 50 km/h  |  >20 km -> 70 km/h
	speed_kmh = np.full_like(dist_km, 50.0)                     # riempimento di default della matrice delle velocità
	speed_kmh[dist_km <= 5.0] = 30.0                            # urbano
	speed_kmh[dist_km > 20.0] = 70.0                            # extraurbano

	# Evita divisione per zero sulla diagonale (il tempo lì lo forziamo a 0)
	np.fill_diagonal(speed_kmh, 1.0)

	# 3.1) tempo = (distanza / velocità) * 60  --> minuti
	time_min = (dist_km / speed_kmh) * 60.0

	# Diagonale a 0 (stesso punto)
	np.fill_diagonal(time_min, 0.0)

	# 4.1) Arrotondamento a interi (meglio CEIL per non sottostimare i tempi)
	time_mat_min = np.ceil(time_min).astype(int)      # matrice delle distanze trasformate in minuti di percorrenza

	# cambio del tipo di variabile per risparmiare memoria
	distance_matrix_metri = distance_matrix_metri.astype(np.uint16)
	time_mat_min = time_mat_min.astype(np.uint16)


	# mapping che serve a risalire dall'indice al location_id originale
	index_to_location_id = points['location_id'].to_list()
	location_id_to_index = {int(lid): i for i, lid in enumerate(points['location_id'])}

	return distance_matrix_metri, time_mat_min, index_to_location_id, location_id_to_index


try to create unique files of matrixes in order to not saturate the ram

In [ ]:

# with open('utility_memory/distance_matrix_OND.pkl', 'wb') as f:
#     pickle.dump(distance_matrix_creation('OND'), f)

# with open('utility_memory/distance_matrix_AS.pkl', 'wb') as f:
#     pickle.dump(distance_matrix_creation('AS'), f)

# with open('utility_memory/distance_matrix_ON.pkl', 'wb') as f:
#     pickle.dump(distance_matrix_creation('ON'), f)

# with open('utility_memory/full_distance_matrix.pkl', 'rb') as f:
#     full_distance_matrix = pickle.load(f)
#     # vedi il tipo di variabile presente all'interno di distance_matrix_metri e time_mat_min
#     print(type(full_distance_matrix[0][0][0]))  # tipo del primo elemento della matrice
#     print(type(full_distance_matrix[1][0][0]))  # tipo del primo elemento della matrice


## TSP con OR-Tools

In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def solve_tsp(distance_matrix_metri, time_mat_min, index_to_location_id=None, time_limit_seconds=30, minutes_per_delivery=10):
    """
    Risolve il TSP (1 veicolo, ritorno al depot) usando la distance_matrix in input_data.
    Ritorna: dict con route in indici, in location_id e distanza totale in metri.
    """
    distance_matrix = time_mat_min
    num_vehicles = 1          # deve essere 1 per TSP
    depot_index = 0           # depot sempre in posizione 0

    # 1) Manager e modello
    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), num_vehicles, depot_index)
    routing = pywrapcp.RoutingModel(manager)

    # 2) Callback costi (distanze in minuti, interi)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node   = manager.IndexToNode(to_index)
        return int(distance_matrix[from_node][to_node])

    transit_cb_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb_index)

    # 3) Parametri di ricerca
    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_params.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_params.time_limit.FromSeconds(int(time_limit_seconds))

    # 4) Risoluzione
    solution = routing.SolveWithParameters(search_params)
    if solution is None:
        # se non ho trovato una soluzione aumento il time limit e riprovo
        search_params.time_limit.FromSeconds(int(time_limit_seconds)*3)
        solution = routing.SolveWithParameters(search_params)
        if solution is None:
            raise RuntimeError("Nessuna soluzione trovata (aumenta il time_limit_seconds).")
        

    # 5) Estrazione del tour
    route_indices = []
    travel_time_min = 0
    index = routing.Start(0)
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route_indices.append(node)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        travel_time_min += routing.GetArcCostForVehicle(previous_index, index, 0)
    # chiudi il ciclo con il depot finale
    route_indices.append(manager.IndexToNode(index))

    # # 6) Mapping (se fornito): indici -> location_id
    # route_location_ids = None
    # if index_to_location_id is not None:
    #     route_location_ids = [index_to_location_id[i] for i in route_indices]

    # 7) Stampa leggibile
    def fmt_route(ids_or_idx):
        return " -> ".join(map(str, ids_or_idx))

    
    # print("TSP – sequenza di nodi (indici):")
    # print(fmt_route(route_indices))
    # if route_location_ids is not None:
    #     print("\nTSP – sequenza di location_id:")
    #     print(fmt_route(route_location_ids))
    

    dist_m = np.asarray(distance_matrix_metri, dtype=float)
    # Somma delle distanze sui successivi archi del tour
    total_distance_m = 0.0

    # Somma archi consecutivi
    for i, j in zip(route_indices[:-1], route_indices[1:]):
        total_distance_m += dist_m[i, j]

    # tempo per lo scarico merci (minuti) - 10 minuti per consegna
    unloading_time = (len(route_indices)-2)*minutes_per_delivery

    #print(f"\nTempo di percorrenza del percorso: {travel_time_min:,} minuti -> {travel_time_min/60:.2f} ore")
    #print(f"\nTempo per effettuare le {len(route_location_ids)-2} (consegne): {unloading_time} minuti -> {unloading_time/60:.2f} ore")
    #print(f"\nTempo totale stimato (percorrenza + consegne): {travel_time_min + unloading_time} minuti -> {(travel_time_min + unloading_time)/60:.2f} ore")

    return {
        # se serve è la lista ordinata degli index dei delivery points -> "route_indices": route_indices,
        # se serve è la lista ordinata degli location_id -> "route_location_ids": route_location_ids,
        "total_distance_m": total_distance_m,         # in metri
        "num_deliveries": len(route_indices)-2,       # escludi depot iniziale e finale
        "travel_time_min": travel_time_min,
        "unloading_time_min": unloading_time,
        "total_delivery_time": travel_time_min + unloading_time
    }



# Calcolo delle performances

## Esecuzione del routing dato un cluster specifico

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_weekday_name(weekday_num):
    return ["Lunedì","Martedì","Mercoledì","Giovedì","Venerdì","Sabato","Domenica"][weekday_num]

def single_cluster_stats_with_cache(
    cluster_location_ids,
    time_limit = 7,                   # time limit per ogni TSP giornaliero
    cache: dict | None = None,        # cache riutilizzabile tra chiamate
    # return_cache = False,             # se True ritorna anche la cache aggiornata
    verbose: bool = False,            # per stampare i dati per debugging
    max_workers: int | None = None,   # parallelismo: None=default
):
    route_cache = cache if cache is not None else {}

    # 1) Filtra il delivery_data al cluster (assumi is_event già True e delivery_date già date)
    dd = delivery_data[delivery_data['location_id'].isin(set(cluster_location_ids))].copy()

    # 2) Costruisci UNA SOLA VOLTA le matrici del cluster
    with open('utility_memory/full_distance_matrix.pkl', 'rb') as f:
        full_distance_matrix = pickle.load(f)
    distance_matrix_metri, time_mat_min, index_to_location_id, location_id_to_index = full_distance_matrix
    full_mat_min = np.asarray(time_mat_min, dtype=float)          # minuti
    full_mat_meters = np.asarray(distance_matrix_metri, dtype=float)  # metri
    depot_global_index = 0  # depot sempre in posizione 0

    # 3) Mappa date -> key (set ordinato di location_id); raccogli i set unici non in cache
    date_to_key: dict = {}
    unique_keys_needed: set = set()

    for the_date, df_day in dd.groupby('delivery_date', sort=True):
        todays_ids = df_day['location_id'].unique().tolist()
        if not todays_ids:
            continue
        key = tuple(sorted(int(x) for x in todays_ids))
        date_to_key[the_date] = key
        if key not in route_cache:
            unique_keys_needed.add(key)

    if verbose:
        print(f"Set unici da risolvere (non in cache): {len(unique_keys_needed)}")

    # 4) Prepara i job per i soli set mancanti (costruisci le sub-matrici una volta qui)
    jobs = []
    for key in unique_keys_needed:
        day_idx_global = [int(location_id_to_index[lid]) for lid in key]
        index_map = [depot_global_index] + day_idx_global
        sub_mat_min = full_mat_min[np.ix_(index_map, index_map)]
        sub_mat_meters = full_mat_meters[np.ix_(index_map, index_map)]
        sub_index_to_location_id = [index_to_location_id[i] for i in index_map]

        jobs.append({
            "key": key,
            "sub_mat_meters": sub_mat_meters.tolist(),
            "sub_mat_min": sub_mat_min.tolist(),
            "sub_index_to_location_id": sub_index_to_location_id
        })

    # 5) Esegui i TSP dei set unici in PARALLELO (thread)
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        for job in jobs:
            fut = ex.submit(
                solve_tsp,
                job["sub_mat_meters"],
                job["sub_mat_min"],
                job["sub_index_to_location_id"],
                time_limit,                      # secondi di time limit per il TSP giornaliero
                10                                # minuti per consegna
            )
            futures.append((job["key"], fut))

        for key, fut in futures:
            result = fut.result()  # se c’è errore, esplode qui (utile per debug)
            route_cache[key] = {
                "total_delivery_time": float(result["total_delivery_time"]),
                "distance_m": float(result["total_distance_m"]),
                "num_deliveries": result["num_deliveries"],
                "travel_time_min": float(result["travel_time_min"]),
                "unloading_time_min": float(result["unloading_time_min"]),
            }
            if verbose:
                print(f"Set {key} risolto in parallelo -> {route_cache[key]['total_delivery_time']:.1f} min")

    # 6) Costruisci le righe giornaliere usando SEMPRE la cache (ora completa)
    rows = []
    for the_date, key in sorted(date_to_key.items(), key=lambda x: x[0]):
        entry = route_cache[key]
        rows.append({
            "delivery_date": the_date,
            "weekday": the_date.weekday(),
            "weekday_name": get_weekday_name(the_date.weekday()),
            "total_delivery_time": entry["total_delivery_time"],
            "distance_m": entry["distance_m"],
            "num_deliveries": entry["num_deliveries"],
            "travel_time_min": entry["travel_time_min"],
            "unloading_time_min": entry["unloading_time_min"],
            "quantity_delivered": dd[dd['delivery_date'] == the_date]['quantity'].sum()
        })

    if not rows:
        return (None, None)

    # 7) Output DataFrame ordinati
    daily_results = (
        pd.DataFrame(rows)
        .sort_values("delivery_date")
        .reset_index(drop=True)
    )

    # Calcolare quante volte il fattorino supera le 8 ore (480 minuti) per giorno della settimana
    daily_results['overtime'] = daily_results['total_delivery_time'] > shift_duration

    # Calcolare la media del tempo extra (solo dove supera 8 ore) per weekday
    daily_results['overtime_minutes'] = np.where(daily_results['overtime'], daily_results['total_delivery_time'] - shift_duration, 0)



    stats_df = (
        daily_results
        .groupby(["weekday", "weekday_name"], as_index=False)
        .agg(
            min_minutes=("total_delivery_time", "min"),
            max_minutes=("total_delivery_time", "max"),
            mean_minutes=("total_delivery_time", "mean"),
            min_distance_m=("distance_m", "min"),
            max_distance_m=("distance_m", "max"),
            mean_distance_m=("distance_m", "mean"),
            min_num_deliveries=("num_deliveries", "min"),
            max_num_deliveries=("num_deliveries", "max"),
            mean_num_deliveries=("num_deliveries", "mean"),
            min_travel_time_min=("travel_time_min", "min"),
            max_travel_time_min=("travel_time_min", "max"),
            mean_travel_time_min=("travel_time_min", "mean"),
            min_unloading_time_min=("unloading_time_min", "min"),
            max_unloading_time_min=("unloading_time_min", "max"),
            mean_unloading_time_min=("unloading_time_min", "mean"),
            n_days=("total_delivery_time", "count"),
            n_overtime_days=("overtime", "sum"),
            max_overtime_minutes=("overtime_minutes", "max"),
            mean_overtime_minutes=("overtime_minutes", lambda x: x[x > 0].mean() if (x > 0).any() else 0),
            min_num_packages=("quantity_delivered", "min"),
            max_num_packages=("quantity_delivered", "max"),
            mean_num_packages=("quantity_delivered", "mean")
        )
        .sort_values("weekday")
        .rename(columns={"weekday_name": "weekday"})
        [["weekday", "min_minutes", "max_minutes", "mean_minutes",
          "min_distance_m", "max_distance_m", "mean_distance_m",
          "min_num_deliveries", "max_num_deliveries", "mean_num_deliveries",
          "min_travel_time_min", "max_travel_time_min", "mean_travel_time_min",
          "min_unloading_time_min", "max_unloading_time_min", "mean_unloading_time_min",
          "n_days", "n_overtime_days", "max_overtime_minutes", "mean_overtime_minutes",
          "min_num_packages", "max_num_packages", "mean_num_packages"]]
        .reset_index(drop=True)
    )

    return stats_df, daily_results


### prova per un singolo cluster

In [ ]:
# stats_df, daily_results = single_cluster_stats_with_cache(
#                 [3836, 3840, 3845, 3851, 3876, 3877, 3881, 3888, 3919, 4730, 4738, 4772, 4773, 4774, 4775, 4776, 4777, 4778, 4779, 4780, 4781, 4782, 4783, 4785, 4786, 4787, 6578, 6598, 6682, 6847, 6971, 6972, 7030, 7104, 7170, 7208, 7210, 7219, 7220, 7222, 7237, 7255, 7269, 7270, 7282, 7290, 7301, 7332, 7333, 7349, 7351, 7394, 7409, 7455, 7995, 9548, 9549, 9756, 12994, 14835, 14953, 16055, 16072, 16278, 16287],
#                 3)

# stats_df

## Metodo principale per il calcolo delle performances di un gruppo di clusters

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def calc_clusters_stats(
    clusters: list[list[int]],
    time_limit: int = 3,
    parallel: bool = False,
    max_workers: int | None = None,
    verbose: bool = False,
):
    """
    Calcola le performances per più cluster e
    ritorna un unico DataFrame con colonne:
    ['cluster','weekday','min_minutes','max_minutes','mean_minutes',
     'min_distance_m','max_distance_m','mean_distance_m','min_num_deliveries', ...]
    dove ogni riga rappresenta (Cluster, Weekday).

    Se desideri una riga sola per cluster aggregando su tutti i weekday,
    vedi il blocco commentato alla fine.
    """
    def _job(name: str, loc_ids: list[int]):
        # Deduplica preservando ordine
        loc_ids = list(dict.fromkeys(loc_ids))
        stats_df, _daily_results = single_cluster_stats_with_cache(
            cluster_location_ids=loc_ids,
            time_limit=time_limit,
            cache=None,
            verbose=verbose
        )
        # aggiunge il nome cluster
        stats_df = stats_df.copy()
        stats_df.insert(0, "cluster", name)
        # rinomina eventuale colonna duplicata 'weekday' (stringa) in 'weekday_name' se necessario
        cols = list(stats_df.columns)
        if cols.count("weekday") == 2:
            # tipicamente stats_df ha ['weekday','min...'] già pulite; ma se ci fosse il nome,
            # lo rinominiamo per chiarezza
            stats_df.columns = ["cluster", "weekday", "weekday_name"] + cols[3:]
        return stats_df

    pieces = []
    if parallel:
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futs = []
            for i, loc_ids in enumerate(clusters, start=1):
                name = f"Cluster {i}"
                futs.append(ex.submit(_job, name, loc_ids))
            for f in as_completed(futs):
                pieces.append(f.result())
    else:
        for i, loc_ids in enumerate(clusters, start=1):
            name = f"Cluster {i}"
            pieces.append(_job(name, loc_ids))

    if not pieces:
        return pd.DataFrame()

    out = pd.concat(pieces, ignore_index=True)

    # Ordina in modo leggibile
    order_cols = [c for c in [
        "cluster", "weekday", "weekday_name",
        "min_minutes", "max_minutes", "mean_minutes",
        "min_distance_m", "max_distance_m", "mean_distance_m",
        "min_num_deliveries", "max_num_deliveries", "mean_num_deliveries",
        "min_travel_time_min", "max_travel_time_min", "mean_travel_time_min",
        "min_unloading_time_min", "max_unloading_time_min", "mean_unloading_time_min",
        "n_days", "n_overtime_days", "max_overtime_minutes", "mean_overtime_minutes",
        "min_num_packages", "max_num_packages", "mean_num_packages"
    ] if c in out.columns]
    out = out[order_cols].sort_values(["cluster", "weekday"]).reset_index(drop=True)
    return out


# Calcolo performances ott-nov-dic (OND)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_weekday_name(weekday_num):
    return ["Lunedì","Martedì","Mercoledì","Giovedì","Venerdì","Sabato","Domenica"][weekday_num]

def single_cluster_stats_with_cache_OND(
    cluster_location_ids,
    time_limit = 7,                   # time limit per ogni TSP giornaliero
    cache: dict | None = None,        # cache riutilizzabile tra chiamate
    # return_cache = False,             # se True ritorna anche la cache aggiornata
    verbose: bool = False,            # per stampare i dati per debugging
    max_workers: int | None = None,   # parallelismo: None=default
):
    route_cache = cache if cache is not None else {}

    # 1) Filtra il delivery_data al cluster (assumi is_event già True e delivery_date già date)
    dd = delivery_data[delivery_data['location_id'].isin(set(cluster_location_ids))].copy()
    # Filtra per ordini solo a ottobre, novembre o dicembre
    dd = dd[dd['delivery_date'].map(lambda d: d.month in {10, 11, 12})].copy()

    # 2) Costruisci UNA SOLA VOLTA le matrici del cluster
    with open('utility_memory/distance_matrix_OND.pkl', 'rb') as f:
        full_distance_matrix = pickle.load(f)
    distance_matrix_metri, time_mat_min, index_to_location_id, location_id_to_index = full_distance_matrix
    full_mat_min = np.asarray(time_mat_min, dtype=float)          # minuti
    full_mat_meters = np.asarray(distance_matrix_metri, dtype=float)  # metri
    depot_global_index = 0  # depot sempre in posizione 0

    # 3) Mappa date -> key (set ordinato di location_id); raccogli i set unici non in cache
    date_to_key: dict = {}
    unique_keys_needed: set = set()

    for the_date, df_day in dd.groupby('delivery_date', sort=True):
        todays_ids = df_day['location_id'].unique().tolist()
        if not todays_ids:
            continue
        key = tuple(sorted(int(x) for x in todays_ids))
        date_to_key[the_date] = key
        if key not in route_cache:
            unique_keys_needed.add(key)

    if verbose:
        print(f"Set unici da risolvere (non in cache): {len(unique_keys_needed)}")

    # 4) Prepara i job per i soli set mancanti (costruisci le sub-matrici una volta qui)
    jobs = []
    for key in unique_keys_needed:
        day_idx_global = [int(location_id_to_index[lid]) for lid in key]
        index_map = [depot_global_index] + day_idx_global
        sub_mat_min = full_mat_min[np.ix_(index_map, index_map)]
        sub_mat_meters = full_mat_meters[np.ix_(index_map, index_map)]
        sub_index_to_location_id = [index_to_location_id[i] for i in index_map]

        jobs.append({
            "key": key,
            "sub_mat_meters": sub_mat_meters.tolist(),
            "sub_mat_min": sub_mat_min.tolist(),
            "sub_index_to_location_id": sub_index_to_location_id
        })

    # 5) Esegui i TSP dei set unici in PARALLELO (thread)
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        for job in jobs:
            fut = ex.submit(
                solve_tsp,
                job["sub_mat_meters"],
                job["sub_mat_min"],
                job["sub_index_to_location_id"],
                time_limit,                      # secondi di time limit per il TSP giornaliero
                10                                # minuti per consegna
            )
            futures.append((job["key"], fut))

        for key, fut in futures:
            result = fut.result()  # se c’è errore, esplode qui (utile per debug)
            route_cache[key] = {
                "total_delivery_time": float(result["total_delivery_time"]),
                "distance_m": float(result["total_distance_m"]),
                "num_deliveries": result["num_deliveries"],
                "travel_time_min": float(result["travel_time_min"]),
                "unloading_time_min": float(result["unloading_time_min"]),
            }
            if verbose:
                print(f"Set {key} risolto in parallelo -> {route_cache[key]['total_delivery_time']:.1f} min")

    # 6) Costruisci le righe giornaliere usando SEMPRE la cache (ora completa)
    rows = []
    for the_date, key in sorted(date_to_key.items(), key=lambda x: x[0]):
        entry = route_cache[key]
        rows.append({
            "delivery_date": the_date,
            "weekday": the_date.weekday(),
            "weekday_name": get_weekday_name(the_date.weekday()),
            "total_delivery_time": entry["total_delivery_time"],
            "distance_m": entry["distance_m"],
            "num_deliveries": entry["num_deliveries"],
            "travel_time_min": entry["travel_time_min"],
            "unloading_time_min": entry["unloading_time_min"],
            "quantity_delivered": dd[dd['delivery_date'] == the_date]['quantity'].sum()
        })

    if not rows:
        return (None, None)

    # 7) Output DataFrame ordinati
    daily_results = (
        pd.DataFrame(rows)
        .sort_values("delivery_date")
        .reset_index(drop=True)
    )

    # Calcolare quante volte il fattorino supera le 8 ore (480 minuti) per giorno della settimana
    daily_results['overtime'] = daily_results['total_delivery_time'] > shift_duration

    # Calcolare la media del tempo extra (solo dove supera 8 ore) per weekday
    daily_results['overtime_minutes'] = np.where(daily_results['overtime'], daily_results['total_delivery_time'] - shift_duration, 0)



    stats_df = (
        daily_results
        .groupby(["weekday", "weekday_name"], as_index=False)
        .agg(
            min_minutes=("total_delivery_time", "min"),
            max_minutes=("total_delivery_time", "max"),
            mean_minutes=("total_delivery_time", "mean"),
            min_distance_m=("distance_m", "min"),
            max_distance_m=("distance_m", "max"),
            mean_distance_m=("distance_m", "mean"),
            min_num_deliveries=("num_deliveries", "min"),
            max_num_deliveries=("num_deliveries", "max"),
            mean_num_deliveries=("num_deliveries", "mean"),
            min_travel_time_min=("travel_time_min", "min"),
            max_travel_time_min=("travel_time_min", "max"),
            mean_travel_time_min=("travel_time_min", "mean"),
            min_unloading_time_min=("unloading_time_min", "min"),
            max_unloading_time_min=("unloading_time_min", "max"),
            mean_unloading_time_min=("unloading_time_min", "mean"),
            n_days=("total_delivery_time", "count"),
            n_overtime_days=("overtime", "sum"),
            max_overtime_minutes=("overtime_minutes", "max"),
            mean_overtime_minutes=("overtime_minutes", lambda x: x[x > 0].mean() if (x > 0).any() else 0),
            min_num_packages=("quantity_delivered", "min"),
            max_num_packages=("quantity_delivered", "max"),
            mean_num_packages=("quantity_delivered", "mean")
        )
        .sort_values("weekday")
        .rename(columns={"weekday_name": "weekday"})
        [["weekday", "min_minutes", "max_minutes", "mean_minutes",
          "min_distance_m", "max_distance_m", "mean_distance_m",
          "min_num_deliveries", "max_num_deliveries", "mean_num_deliveries",
          "min_travel_time_min", "max_travel_time_min", "mean_travel_time_min",
          "min_unloading_time_min", "max_unloading_time_min", "mean_unloading_time_min",
          "n_days", "n_overtime_days", "max_overtime_minutes", "mean_overtime_minutes",
          "min_num_packages", "max_num_packages", "mean_num_packages"]]
        .reset_index(drop=True)
    )

    return stats_df, daily_results


In [ ]:
# stats_df, daily_results = single_cluster_stats_with_cache_OND(
#                 [3836, 3840, 3845, 3851, 3876, 3877, 3881, 3888, 3919, 4730, 4738, 4772, 4773, 4774, 4775, 4776, 4777, 4778, 4779, 4780, 4781, 4782, 4783, 4785, 4786, 4787, 6578, 6598, 6682, 6847, 6971, 6972, 7030, 7104, 7170, 7208, 7210, 7219, 7220, 7222, 7237, 7255, 7269, 7270, 7282, 7290, 7301, 7332, 7333, 7349, 7351, 7394, 7409, 7455, 7995, 9548, 9549, 9756, 12994, 14835, 14953, 16055, 16072, 16278, 16287],
#                 3)

# stats_df

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def calc_clusters_stats_OND(
    clusters: list[list[int]],
    time_limit: int = 3,
    parallel: bool = False,
    max_workers: int | None = None,
    verbose: bool = False,
):
    """
    Calcola le performances per più cluster e
    ritorna un unico DataFrame con colonne:
    ['cluster','weekday','min_minutes','max_minutes','mean_minutes',
     'min_distance_m','max_distance_m','mean_distance_m','min_num_deliveries', ...]
    dove ogni riga rappresenta (Cluster, Weekday).

    Se desideri una riga sola per cluster aggregando su tutti i weekday,
    vedi il blocco commentato alla fine.
    """
    def _job(name: str, loc_ids: list[int]):
        # Deduplica preservando ordine
        loc_ids = list(dict.fromkeys(loc_ids))
        stats_df, _daily_results = single_cluster_stats_with_cache_OND(
            cluster_location_ids=loc_ids,
            time_limit=time_limit,
            cache=None,
            verbose=verbose
        )
        # aggiunge il nome cluster
        stats_df = stats_df.copy()
        stats_df.insert(0, "cluster", name)
        # rinomina eventuale colonna duplicata 'weekday' (stringa) in 'weekday_name' se necessario
        cols = list(stats_df.columns)
        if cols.count("weekday") == 2:
            # tipicamente stats_df ha ['weekday','min...'] già pulite; ma se ci fosse il nome,
            # lo rinominiamo per chiarezza
            stats_df.columns = ["cluster", "weekday", "weekday_name"] + cols[3:]
        return stats_df

    pieces = []
    if parallel:
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futs = []
            for i, loc_ids in enumerate(clusters, start=1):
                name = f"Cluster {i}"
                futs.append(ex.submit(_job, name, loc_ids))
            for f in as_completed(futs):
                pieces.append(f.result())
    else:
        for i, loc_ids in enumerate(clusters, start=1):
            name = f"Cluster {i}"
            pieces.append(_job(name, loc_ids))

    if not pieces:
        return pd.DataFrame()

    out = pd.concat(pieces, ignore_index=True)

    # Ordina in modo leggibile
    order_cols = [c for c in [
        "cluster", "weekday", "weekday_name",
        "min_minutes", "max_minutes", "mean_minutes",
        "min_distance_m", "max_distance_m", "mean_distance_m",
        "min_num_deliveries", "max_num_deliveries", "mean_num_deliveries",
        "min_travel_time_min", "max_travel_time_min", "mean_travel_time_min",
        "min_unloading_time_min", "max_unloading_time_min", "mean_unloading_time_min",
        "n_days", "n_overtime_days", "max_overtime_minutes", "mean_overtime_minutes",
        "min_num_packages", "max_num_packages", "mean_num_packages"
    ] if c in out.columns]
    out = out[order_cols].sort_values(["cluster", "weekday"]).reset_index(drop=True)
    return out

# Calcolo performances agosto-settembre

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_weekday_name(weekday_num):
    return ["Lunedì","Martedì","Mercoledì","Giovedì","Venerdì","Sabato","Domenica"][weekday_num]

def single_cluster_stats_with_cache_AS(
    cluster_location_ids,
    time_limit = 7,                   # time limit per ogni TSP giornaliero
    cache: dict | None = None,        # cache riutilizzabile tra chiamate
    # return_cache = False,             # se True ritorna anche la cache aggiornata
    verbose: bool = False,            # per stampare i dati per debugging
    max_workers: int | None = None,   # parallelismo: None=default
):
    route_cache = cache if cache is not None else {}

    # 1) Filtra il delivery_data_AS al cluster (assumi is_event già True e delivery_date già date)
    dd = delivery_data_AS[delivery_data_AS['location_id'].isin(set(cluster_location_ids))].copy()

    # 2) Costruisci UNA SOLA VOLTA le matrici del cluster
    with open('utility_memory/distance_matrix_AS.pkl', 'rb') as f:
        distance_matrix_AS = pickle.load(f)
    distance_matrix_metri, time_mat_min, index_to_location_id, location_id_to_index = distance_matrix_AS
    full_mat_min = np.asarray(time_mat_min, dtype=float)          # minuti
    full_mat_meters = np.asarray(distance_matrix_metri, dtype=float)  # metri
    depot_global_index = 0  # depot sempre in posizione 0

    # 3) Mappa date -> key (set ordinato di location_id); raccogli i set unici non in cache
    date_to_key: dict = {}
    unique_keys_needed: set = set()

    for the_date, df_day in dd.groupby('delivery_date', sort=True):
        todays_ids = df_day['location_id'].unique().tolist()
        if not todays_ids:
            continue
        key = tuple(sorted(int(x) for x in todays_ids))
        date_to_key[the_date] = key
        if key not in route_cache:
            unique_keys_needed.add(key)

    if verbose:
        print(f"Set unici da risolvere (non in cache): {len(unique_keys_needed)}")

    # 4) Prepara i job per i soli set mancanti (costruisci le sub-matrici una volta qui)
    jobs = []
    for key in unique_keys_needed:
        day_idx_global = [int(location_id_to_index[lid]) for lid in key]
        index_map = [depot_global_index] + day_idx_global
        sub_mat_min = full_mat_min[np.ix_(index_map, index_map)]
        sub_mat_meters = full_mat_meters[np.ix_(index_map, index_map)]
        sub_index_to_location_id = [index_to_location_id[i] for i in index_map]

        jobs.append({
            "key": key,
            "sub_mat_meters": sub_mat_meters.tolist(),
            "sub_mat_min": sub_mat_min.tolist(),
            "sub_index_to_location_id": sub_index_to_location_id
        })

    # 5) Esegui i TSP dei set unici in PARALLELO (thread)
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        for job in jobs:
            fut = ex.submit(
                solve_tsp,
                job["sub_mat_meters"],
                job["sub_mat_min"],
                job["sub_index_to_location_id"],
                time_limit,                      # secondi di time limit per il TSP giornaliero
                10                                # minuti per consegna
            )
            futures.append((job["key"], fut))

        for key, fut in futures:
            result = fut.result()  # se c’è errore, esplode qui (utile per debug)
            route_cache[key] = {
                "total_delivery_time": float(result["total_delivery_time"]),
                "distance_m": float(result["total_distance_m"]),
                "num_deliveries": result["num_deliveries"],
                "travel_time_min": float(result["travel_time_min"]),
                "unloading_time_min": float(result["unloading_time_min"]),
            }
            if verbose:
                print(f"Set {key} risolto in parallelo -> {route_cache[key]['total_delivery_time']:.1f} min")

    # 6) Costruisci le righe giornaliere usando SEMPRE la cache (ora completa)
    rows = []
    for the_date, key in sorted(date_to_key.items(), key=lambda x: x[0]):
        entry = route_cache[key]
        rows.append({
            "delivery_date": the_date,
            "weekday": the_date.weekday(),
            "weekday_name": get_weekday_name(the_date.weekday()),
            "total_delivery_time": entry["total_delivery_time"],
            "distance_m": entry["distance_m"],
            "num_deliveries": entry["num_deliveries"],
            "travel_time_min": entry["travel_time_min"],
            "unloading_time_min": entry["unloading_time_min"],
            "quantity_delivered": dd[dd['delivery_date'] == the_date]['quantity'].sum()
        })

    if not rows:
        return (None, None)

    # 7) Output DataFrame ordinati
    daily_results = (
        pd.DataFrame(rows)
        .sort_values("delivery_date")
        .reset_index(drop=True)
    )

    # Calcolare quante volte il fattorino supera le 8 ore (480 minuti) per giorno della settimana
    daily_results['overtime'] = daily_results['total_delivery_time'] > shift_duration

    # Calcolare la media del tempo extra (solo dove supera 8 ore) per weekday
    daily_results['overtime_minutes'] = np.where(daily_results['overtime'], daily_results['total_delivery_time'] - shift_duration, 0)



    stats_df = (
        daily_results
        .groupby(["weekday", "weekday_name"], as_index=False)
        .agg(
            min_minutes=("total_delivery_time", "min"),
            max_minutes=("total_delivery_time", "max"),
            mean_minutes=("total_delivery_time", "mean"),
            min_distance_m=("distance_m", "min"),
            max_distance_m=("distance_m", "max"),
            mean_distance_m=("distance_m", "mean"),
            min_num_deliveries=("num_deliveries", "min"),
            max_num_deliveries=("num_deliveries", "max"),
            mean_num_deliveries=("num_deliveries", "mean"),
            min_travel_time_min=("travel_time_min", "min"),
            max_travel_time_min=("travel_time_min", "max"),
            mean_travel_time_min=("travel_time_min", "mean"),
            min_unloading_time_min=("unloading_time_min", "min"),
            max_unloading_time_min=("unloading_time_min", "max"),
            mean_unloading_time_min=("unloading_time_min", "mean"),
            n_days=("total_delivery_time", "count"),
            n_overtime_days=("overtime", "sum"),
            max_overtime_minutes=("overtime_minutes", "max"),
            mean_overtime_minutes=("overtime_minutes", lambda x: x[x > 0].mean() if (x > 0).any() else 0),
            min_num_packages=("quantity_delivered", "min"),
            max_num_packages=("quantity_delivered", "max"),
            mean_num_packages=("quantity_delivered", "mean")
        )
        .sort_values("weekday")
        .rename(columns={"weekday_name": "weekday"})
        [["weekday", "min_minutes", "max_minutes", "mean_minutes",
          "min_distance_m", "max_distance_m", "mean_distance_m",
          "min_num_deliveries", "max_num_deliveries", "mean_num_deliveries",
          "min_travel_time_min", "max_travel_time_min", "mean_travel_time_min",
          "min_unloading_time_min", "max_unloading_time_min", "mean_unloading_time_min",
          "n_days", "n_overtime_days", "max_overtime_minutes", "mean_overtime_minutes",
          "min_num_packages", "max_num_packages", "mean_num_packages"]]
        .reset_index(drop=True)
    )

    return stats_df, daily_results


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def calc_clusters_stats_AS(
    clusters: list[list[int]],
    time_limit: int = 3,
    parallel: bool = False,
    max_workers: int | None = None,
    verbose: bool = False,
):
    """
    Calcola le performances per più cluster e
    ritorna un unico DataFrame con colonne:
    ['cluster','weekday','min_minutes','max_minutes','mean_minutes',
     'min_distance_m','max_distance_m','mean_distance_m','min_num_deliveries', ...]
    dove ogni riga rappresenta (Cluster, Weekday).

    Se desideri una riga sola per cluster aggregando su tutti i weekday,
    vedi il blocco commentato alla fine.
    """
    def _job(name: str, loc_ids: list[int]):
        # Deduplica preservando ordine
        loc_ids = list(dict.fromkeys(loc_ids))
        stats_df, _daily_results = single_cluster_stats_with_cache_AS(
            cluster_location_ids=loc_ids,
            time_limit=time_limit,
            cache=None,
            verbose=verbose
        )
        # aggiunge il nome cluster
        stats_df = stats_df.copy()
        stats_df.insert(0, "cluster", name)
        # rinomina eventuale colonna duplicata 'weekday' (stringa) in 'weekday_name' se necessario
        cols = list(stats_df.columns)
        if cols.count("weekday") == 2:
            # tipicamente stats_df ha ['weekday','min...'] già pulite; ma se ci fosse il nome,
            # lo rinominiamo per chiarezza
            stats_df.columns = ["cluster", "weekday", "weekday_name"] + cols[3:]
        return stats_df

    pieces = []
    if parallel:
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futs = []
            for i, loc_ids in enumerate(clusters, start=1):
                name = f"Cluster {i}"
                futs.append(ex.submit(_job, name, loc_ids))
            for f in as_completed(futs):
                pieces.append(f.result())
    else:
        for i, loc_ids in enumerate(clusters, start=1):
            name = f"Cluster {i}"
            pieces.append(_job(name, loc_ids))

    if not pieces:
        return pd.DataFrame()

    out = pd.concat(pieces, ignore_index=True)

    # Ordina in modo leggibile
    order_cols = [c for c in [
        "cluster", "weekday", "weekday_name",
        "min_minutes", "max_minutes", "mean_minutes",
        "min_distance_m", "max_distance_m", "mean_distance_m",
        "min_num_deliveries", "max_num_deliveries", "mean_num_deliveries",
        "min_travel_time_min", "max_travel_time_min", "mean_travel_time_min",
        "min_unloading_time_min", "max_unloading_time_min", "mean_unloading_time_min",
        "n_days", "n_overtime_days", "max_overtime_minutes", "mean_overtime_minutes",
        "min_num_packages", "max_num_packages", "mean_num_packages"
    ] if c in out.columns]
    out = out[order_cols].sort_values(["cluster", "weekday"]).reset_index(drop=True)
    return out

# Calcolo performances ottobre-novembre

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_weekday_name(weekday_num):
    return ["Lunedì","Martedì","Mercoledì","Giovedì","Venerdì","Sabato","Domenica"][weekday_num]

def single_cluster_stats_with_cache_ON(
    cluster_location_ids,
    time_limit = 7,                   # time limit per ogni TSP giornaliero
    cache: dict | None = None,        # cache riutilizzabile tra chiamate
    # return_cache = False,             # se True ritorna anche la cache aggiornata
    verbose: bool = False,            # per stampare i dati per debugging
    max_workers: int | None = None,   # parallelismo: None=default
):
    route_cache = cache if cache is not None else {}

    # 1) Filtra il delivery_data_ON al cluster (assumi is_event già True e delivery_date già date)
    dd = delivery_data_ON[delivery_data_ON['location_id'].isin(set(cluster_location_ids))].copy()

    # 2) Costruisci UNA SOLA VOLTA le matrici del cluster
    with open('utility_memory/distance_matrix_ON.pkl', 'rb') as f:
        distance_matrix_ON = pickle.load(f)
    distance_matrix_metri, time_mat_min, index_to_location_id, location_id_to_index = distance_matrix_ON
    full_mat_min = np.asarray(time_mat_min, dtype=float)          # minuti
    full_mat_meters = np.asarray(distance_matrix_metri, dtype=float)  # metri
    depot_global_index = 0  # depot sempre in posizione 0

    # 3) Mappa date -> key (set ordinato di location_id); raccogli i set unici non in cache
    date_to_key: dict = {}
    unique_keys_needed: set = set()

    for the_date, df_day in dd.groupby('delivery_date', sort=True):
        todays_ids = df_day['location_id'].unique().tolist()
        if not todays_ids:
            continue
        key = tuple(sorted(int(x) for x in todays_ids))
        date_to_key[the_date] = key
        if key not in route_cache:
            unique_keys_needed.add(key)

    if verbose:
        print(f"Set unici da risolvere (non in cache): {len(unique_keys_needed)}")

    # 4) Prepara i job per i soli set mancanti (costruisci le sub-matrici una volta qui)
    jobs = []
    for key in unique_keys_needed:
        day_idx_global = [int(location_id_to_index[lid]) for lid in key]
        index_map = [depot_global_index] + day_idx_global
        sub_mat_min = full_mat_min[np.ix_(index_map, index_map)]
        sub_mat_meters = full_mat_meters[np.ix_(index_map, index_map)]
        sub_index_to_location_id = [index_to_location_id[i] for i in index_map]

        jobs.append({
            "key": key,
            "sub_mat_meters": sub_mat_meters.tolist(),
            "sub_mat_min": sub_mat_min.tolist(),
            "sub_index_to_location_id": sub_index_to_location_id
        })

    # 5) Esegui i TSP dei set unici in PARALLELO (thread)
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        for job in jobs:
            fut = ex.submit(
                solve_tsp,
                job["sub_mat_meters"],
                job["sub_mat_min"],
                job["sub_index_to_location_id"],
                time_limit,                      # secondi di time limit per il TSP giornaliero
                10                                # minuti per consegna
            )
            futures.append((job["key"], fut))

        for key, fut in futures:
            result = fut.result()  # se c’è errore, esplode qui (utile per debug)
            route_cache[key] = {
                "total_delivery_time": float(result["total_delivery_time"]),
                "distance_m": float(result["total_distance_m"]),
                "num_deliveries": result["num_deliveries"],
                "travel_time_min": float(result["travel_time_min"]),
                "unloading_time_min": float(result["unloading_time_min"]),
            }
            if verbose:
                print(f"Set {key} risolto in parallelo -> {route_cache[key]['total_delivery_time']:.1f} min")

    # 6) Costruisci le righe giornaliere usando SEMPRE la cache (ora completa)
    rows = []
    for the_date, key in sorted(date_to_key.items(), key=lambda x: x[0]):
        entry = route_cache[key]
        rows.append({
            "delivery_date": the_date,
            "weekday": the_date.weekday(),
            "weekday_name": get_weekday_name(the_date.weekday()),
            "total_delivery_time": entry["total_delivery_time"],
            "distance_m": entry["distance_m"],
            "num_deliveries": entry["num_deliveries"],
            "travel_time_min": entry["travel_time_min"],
            "unloading_time_min": entry["unloading_time_min"],
            "quantity_delivered": dd[dd['delivery_date'] == the_date]['quantity'].sum()
        })

    if not rows:
        return (None, None)

    # 7) Output DataFrame ordinati
    daily_results = (
        pd.DataFrame(rows)
        .sort_values("delivery_date")
        .reset_index(drop=True)
    )

    # Calcolare quante volte il fattorino supera le 8 ore (480 minuti) per giorno della settimana
    daily_results['overtime'] = daily_results['total_delivery_time'] > shift_duration

    # Calcolare la media del tempo extra (solo dove supera 8 ore) per weekday
    daily_results['overtime_minutes'] = np.where(daily_results['overtime'], daily_results['total_delivery_time'] - shift_duration, 0)



    stats_df = (
        daily_results
        .groupby(["weekday", "weekday_name"], as_index=False)
        .agg(
            min_minutes=("total_delivery_time", "min"),
            max_minutes=("total_delivery_time", "max"),
            mean_minutes=("total_delivery_time", "mean"),
            min_distance_m=("distance_m", "min"),
            max_distance_m=("distance_m", "max"),
            mean_distance_m=("distance_m", "mean"),
            min_num_deliveries=("num_deliveries", "min"),
            max_num_deliveries=("num_deliveries", "max"),
            mean_num_deliveries=("num_deliveries", "mean"),
            min_travel_time_min=("travel_time_min", "min"),
            max_travel_time_min=("travel_time_min", "max"),
            mean_travel_time_min=("travel_time_min", "mean"),
            min_unloading_time_min=("unloading_time_min", "min"),
            max_unloading_time_min=("unloading_time_min", "max"),
            mean_unloading_time_min=("unloading_time_min", "mean"),
            n_days=("total_delivery_time", "count"),
            n_overtime_days=("overtime", "sum"),
            max_overtime_minutes=("overtime_minutes", "max"),
            mean_overtime_minutes=("overtime_minutes", lambda x: x[x > 0].mean() if (x > 0).any() else 0),
            min_num_packages=("quantity_delivered", "min"),
            max_num_packages=("quantity_delivered", "max"),
            mean_num_packages=("quantity_delivered", "mean")
        )
        .sort_values("weekday")
        .rename(columns={"weekday_name": "weekday"})
        [["weekday", "min_minutes", "max_minutes", "mean_minutes",
          "min_distance_m", "max_distance_m", "mean_distance_m",
          "min_num_deliveries", "max_num_deliveries", "mean_num_deliveries",
          "min_travel_time_min", "max_travel_time_min", "mean_travel_time_min",
          "min_unloading_time_min", "max_unloading_time_min", "mean_unloading_time_min",
          "n_days", "n_overtime_days", "max_overtime_minutes", "mean_overtime_minutes",
          "min_num_packages", "max_num_packages", "mean_num_packages"]]
        .reset_index(drop=True)
    )

    return stats_df, daily_results


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def calc_clusters_stats_ON(
    clusters: list[list[int]],
    time_limit: int = 3,
    parallel: bool = False,
    max_workers: int | None = None,
    verbose: bool = False,
):
    """
    Calcola le performances per più cluster e
    ritorna un unico DataFrame con colonne:
    ['cluster','weekday','min_minutes','max_minutes','mean_minutes',
     'min_distance_m','max_distance_m','mean_distance_m','min_num_deliveries', ...]
    dove ogni riga rappresenta (Cluster, Weekday).

    Se desideri una riga sola per cluster aggregando su tutti i weekday,
    vedi il blocco commentato alla fine.
    """
    def _job(name: str, loc_ids: list[int]):
        # Deduplica preservando ordine
        loc_ids = list(dict.fromkeys(loc_ids))
        stats_df, _daily_results = single_cluster_stats_with_cache_ON(
            cluster_location_ids=loc_ids,
            time_limit=time_limit,
            cache=None,
            verbose=verbose
        )
        # aggiunge il nome cluster
        stats_df = stats_df.copy()
        stats_df.insert(0, "cluster", name)
        # rinomina eventuale colonna duplicata 'weekday' (stringa) in 'weekday_name' se necessario
        cols = list(stats_df.columns)
        if cols.count("weekday") == 2:
            # tipicamente stats_df ha ['weekday','min...'] già pulite; ma se ci fosse il nome,
            # lo rinominiamo per chiarezza
            stats_df.columns = ["cluster", "weekday", "weekday_name"] + cols[3:]
        return stats_df

    pieces = []
    if parallel:
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futs = []
            for i, loc_ids in enumerate(clusters, start=1):
                name = f"Cluster {i}"
                futs.append(ex.submit(_job, name, loc_ids))
            for f in as_completed(futs):
                pieces.append(f.result())
    else:
        for i, loc_ids in enumerate(clusters, start=1):
            name = f"Cluster {i}"
            pieces.append(_job(name, loc_ids))

    if not pieces:
        return pd.DataFrame()

    out = pd.concat(pieces, ignore_index=True)

    # Ordina in modo leggibile
    order_cols = [c for c in [
        "cluster", "weekday", "weekday_name",
        "min_minutes", "max_minutes", "mean_minutes",
        "min_distance_m", "max_distance_m", "mean_distance_m",
        "min_num_deliveries", "max_num_deliveries", "mean_num_deliveries",
        "min_travel_time_min", "max_travel_time_min", "mean_travel_time_min",
        "min_unloading_time_min", "max_unloading_time_min", "mean_unloading_time_min",
        "n_days", "n_overtime_days", "max_overtime_minutes", "mean_overtime_minutes",
        "min_num_packages", "max_num_packages", "mean_num_packages"
    ] if c in out.columns]
    out = out[order_cols].sort_values(["cluster", "weekday"]).reset_index(drop=True)
    return out